# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 5 2023 8:30AM,254301,10,8990139-SD,Eywa Pharma Inc.,Released
1,Jan 5 2023 8:30AM,254306,10,8990148-SD,Eywa Pharma Inc.,Released
2,Jan 5 2023 8:30AM,254310,10,8990153-SD,Eywa Pharma Inc.,Released
3,Jan 5 2023 8:01AM,254319,19,ADV80010055,"AdvaGen Pharma, Ltd",Released
4,Jan 5 2023 8:01AM,254319,19,ADV80010056,"AdvaGen Pharma, Ltd",Released
5,Jan 5 2023 8:01AM,254319,19,ADV80010057,"AdvaGen Pharma, Ltd",Released
6,Jan 5 2023 8:01AM,254319,19,ADV80010058,"AdvaGen Pharma, Ltd",Released
7,Jan 5 2023 8:01AM,254319,19,ADV80010059,"AdvaGen Pharma, Ltd",Released
8,Jan 5 2023 8:01AM,254319,19,ADV80010060,"AdvaGen Pharma, Ltd",Released
9,Jan 5 2023 8:01AM,254319,19,ADV80010061,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,254310,Released,1
13,254312,Released,36
14,254315,Released,1
15,254318,Executing,1
16,254319,Released,50


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254310,NaN,1.0
254312,NaN,36.0
254315,NaN,1.0
254318,1.0,NaN
254319,NaN,50.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254255,1.0,0.0
254265,0.0,1.0
254266,0.0,1.0
254270,0.0,1.0
254282,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254255,1,0
254265,0,1
254266,0,1
254270,0,1
254282,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254255,1,0
1,254265,0,1
2,254266,0,1
3,254270,0,1
4,254282,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254255,1,
1,254265,,1
2,254266,,1
3,254270,,1
4,254282,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd"
53,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC"
54,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc."
55,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc."
91,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd"
97,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC"
98,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,,1
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,,1
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,,1
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",,50
4,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC",1,
5,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",,1
6,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",,36
7,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",,6
8,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",,1
9,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1,
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",50,
4,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC",,1
5,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",1,
6,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36,
7,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6,
8,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",1,
9,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1,
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",50,
4,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,NaN
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,NaN
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1.0,NaN
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",50.0,NaN
4,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,0.0
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,0.0
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1.0,0.0
3,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",50.0,0.0
4,Jan 5 2023 7:50AM,254318,20,"Emersa Waterbox, LLC",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,762917,3.0,0.0
15,762884,39.0,0.0
16,254265,1.0,0.0
19,1780069,73.0,1.0
20,254318,0.0,1.0
21,508581,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,762917,3.0,0.0
1,15,762884,39.0,0.0
2,16,254265,1.0,0.0
3,19,1780069,73.0,1.0
4,20,254318,0.0,1.0
5,21,508581,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,3.0,0.0
1,15,39.0,0.0
2,16,1.0,0.0
3,19,73.0,1.0
4,20,0.0,1.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,3.0
1,15,Released,39.0
2,16,Released,1.0
3,19,Released,73.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,0.0,0.0,0.0,1.0,1.0,0.0
Released,3.0,39.0,1.0,73.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,0.0,0.0,0.0,1.0,1.0,0.0
1,Released,3.0,39.0,1.0,73.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,0.0,0.0,0.0,1.0,1.0,0.0
1,Released,3.0,39.0,1.0,73.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()